In [15]:
!pip install moviepy==1.0.3 "scikit-video>=1.1.11" numpy==1.26.0 'opencv-python>=4.10.0.84' 'diffusers>=0.30.1'

# 需要重启

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
DEPRECATION: omegaconf 2.0.5 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [1]:
import sys
sys.path.append('/mnt/ceph/develop/jiawei/lora_dataset/Keyframe-Extraction-for-video-summarization/src')

from rife_model import load_rife_model, rife_inference_with_latents
from huggingface_hub import hf_hub_download, snapshot_download

/mnt/ceph/develop/jiawei/conda_env/keyframe_extra/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
snapshot_download(repo_id="AlexWortega/RIFE", local_dir="model_rife")


Fetching 2 files: 100%|████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 251.45it/s]


'/mnt/ceph/develop/jiawei/lora_dataset/Keyframe-Extraction-for-video-summarization/src/model_rife'

### 读取视频帧，做一倍插针

In [3]:
import torch
torch.set_grad_enabled(False)
if torch.cuda.is_available():
    torch.backends.cudnn.enabled = True
    torch.backends.cudnn.benchmark = True
    torch.set_default_tensor_type(torch.cuda.HalfTensor)

frame_interpolation_model = load_rife_model("model_rife")

frame_interpolation_model.device()


/mnt/ceph/develop/jiawei/conda_env/keyframe_extra/lib/python3.10/site-packages/torch/__init__.py:614: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:451.)
  _C._set_default_tensor_type(t)


In [4]:
video_path = '/mnt/ceph/develop/jiawei/lora_dataset/speech_data/B站/test_1/cogvideo_00106.mp4'

In [5]:
import cv2
import numpy as np
import utils
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
video_capture = cv2.VideoCapture(video_path)
tot_frame = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))  # Get total frames
pt_frame_data = []

# Read each frame using VideoCapture
while video_capture.isOpened():
    ret, frame = video_capture.read()
    if not ret:
        break
    # BGR to RGB
    frame_rgb = frame[..., ::-1]
    frame_rgb = frame_rgb.copy()
    tensor = torch.from_numpy(frame_rgb).float().to("cpu", non_blocking=True).float() / 255.0
    pt_frame_data.append(
        tensor.permute(2, 0, 1)
    )  # to [c, h, w,]

video_capture.release()  # Release the video after done

pt_frame = torch.from_numpy(np.stack(pt_frame_data))
pt_frame = pt_frame.to(device)
pt_frame = pt_frame.unsqueeze(0)
pt_frame.shape
pt_frame = pt_frame.to(dtype=torch.float16)
pt_frame.dtype

torch.float16

In [6]:
rife_pt_frame = rife_inference_with_latents(frame_interpolation_model,pt_frame )
rife_pt_frame.shape

RIFE inference:   0%|                                                                                           | 0/8 [00:00<?, ?it/s]

samples dtype:torch.float16
samples shape:torch.Size([8, 3, 480, 720])
torch.float16
torch.float16


RIFE inference:   0%|                                                                                           | 0/8 [00:02<?, ?it/s]

torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16


torch.Size([1, 16, 3, 480, 720])

In [7]:
from diffusers.image_processor import VaeImageProcessor
pt_image = rife_pt_frame[0]
pt_image = torch.stack([pt_image[i] for i in range(pt_image.shape[0])])
 
image_np = VaeImageProcessor.pt_to_numpy(pt_image)
image_pil = VaeImageProcessor.numpy_to_pil(image_np)

In [8]:
from diffusers.utils import export_to_video
import math

export_video_path =  '/mnt/ceph/develop/jiawei/lora_dataset/speech_data/B站/test_1/rife_chunk_5.mp4'
export_to_video(image_pil, export_video_path, fps=math.ceil((len(image_pil) - 1) / 6))

'/mnt/ceph/develop/jiawei/lora_dataset/speech_data/B站/test_1/rife_chunk_5.mp4'

In [9]:
import cv2
import math


def read_fps(video_path:str):
    # 打开视频文件
    video_capture = cv2.VideoCapture(video_path)
    
    # 获取视频的帧速率
    fps = int(video_capture.get(cv2.CAP_PROP_FPS))
     
    # 释放视频捕获对象
    video_capture.release()
    
    # 关闭视频文件
    cv2.destroyAllWindows()
    return fps

In [10]:
video_path=f'/mnt/ceph/develop/jiawei/lora_dataset/speech_data/B站/test_1/rife_chunk_5.mp4'

fps = read_fps(video_path)

fps

3